# Ipyreact walkthrough

Welcome to this ipyreact walkthrough!   
The tutorial will be based on a very simple react button to show all the ipyreact features.  

**Content** 
* use the %react cell magic
* write a widget
* style this with CSS
* add parameters to your widgets (traitlets)
* interact with these parameters
* simple traitlet oberservation using `change`
* observe a traitlet and call python function
* observe a traitlet and call JavaScript function
* call python functions from JavaScript
* loading components from external files
* enable hot reloading  
* enable autocompletion in IDEs
* print a message at class initialization

First, we will use the **`%react` magic**  from ipyreact.  
The following line registers the cellmagic:

In [ ]:
%pip install -q ipyreact
# This line is for JupyterLite (if this takes more than 10 seconds, something probably hung, restart the kernel and run this cell again)

In [ ]:
%load_ext ipyreact

In [ ]:
%%react

import * as React from "react";

export default function MyButton() {
    return ( < button > X < /button>);
}

Great, here we can see react code rendering in the jupyter notebook!  
Next, we **convert this into a widget.**  
For that, we need the code in a `_esm` string inside a class that inherits from `ipyreact.Widget`.  
`esm` is short for for EcmaScript module, and thats standard for structuring JavaScript code in reusable components.

In [ ]:
import ipyreact

class MyExampleWidget(ipyreact.Widget):
    _esm = """
    import * as React from "react";

    export default function MyButton() {
        return <button class="mybutton"> X < /button> 
    };
    """

MyExampleWidget()

## HTML elements, children and props
For simple built-in HTML elements that the browser supports, such as button, we can also use the following:

In [ ]:
ipyreact.Widget(_type="button", children=["X"])

In this case, ipyreact simply wraps [React's createElement](https://react.dev/reference/react/createElement) where children can be text, or an ipywidget. If the widget is a ipyreact widget, the react render tree will continue uninterrupted.

The props are passed to the createElement function, and for builtin elements can be common attributes such as `style`, `class` and `title`.

In [ ]:
ipyreact.Widget(_type="button", children=["X"], props={
    "title": "Behaves like a tooltip",
    "style": {"border": "5px solid orange"},
    "class": "mybutton"
})

## Styling

Let's style this with CSS

In [ ]:
from IPython.display import HTML
css_rules = """
.mybutton {
    background-color: yellow;
    border-radius: 10px;
}
"""
HTML("<style>" + css_rules + "</style>")

Note: Loading CSS styles is global and will include every cell of the notebook.   
Also the widgets at the bottom of the notebook will be affected.  

You can also load CSS from a file.  
And you can add extra CSS without overwriting the previous CSS import.

In [ ]:
from IPython.display import HTML
from pathlib import Path
css_rules = Path("styles_orange.css").read_text()
HTML("<style>" + css_rules + "</style>")

Note: When you clear the output of the previous cell, this CSS style will be removed from the notebook.

## Parametizing


Next, we want to **add parameters to the widget.**  
First naive approach: Using f-strings.
That works, but is bad for two reasons:  
1. curly brackets from f-strings will collide with the curly brackets from JavaScript/TypeScript. 
Using curly brackets in an f-string as a normal character can be done by doubling them like this : `{`to `{{`.  
2. The f-string is only interpreted when the class is interpreted. That means it's won't be possible to change the value of `my_message` when interacting with the widget.
   

In [ ]:
import ipyreact

# ❌❌❌  WARNING: THIS CODEBLOCK IS NOT GOOD PRACTICE ❌❌❌ 
class MyExampleWidget(ipyreact.Widget):
    message = "Hello World"
    _esm = f"""
    import * as React from "react";

    export default function MyButton() {{
        return <button> {message} </button> 
    }};"""
MyExampleWidget()

## Parametrizing using props

If you only want to set a value from the Python side, you can use the props to pass data to the component in the frontend.


In [ ]:
import ipyreact
from traitlets import Unicode

class MyExampleWidget(ipyreact.Widget):
    _esm = """
    import * as React from "react";

    export default function MyButton({ message }) {
        return <button> {message} </button>;
    };"""
MyExampleWidget(props={"message": "hi"})

### Forwarding unused props and children
However, with this, we lose the ability to set all the other props like in the example above. We can use the following pattern using [object destructuring](https://developer.mozilla.org/en-US/docs/Web/JavaScript/Reference/Operators/Destructuring_assignment#object_destructuring) by passing all unused props (called `rest` in this example) to the button element. Also, do not forget to pass the children!.

In [ ]:
import ipyreact
from traitlets import Unicode

# ⭐⭐⭐  This is good practice again ⭐⭐⭐
class MyExampleWidget(ipyreact.Widget):
    _esm = """
    import * as React from "react";

    export default function MyButton({ message, children, ...rest }) {
        return <button {...rest}> {[message, ...children]} </button>;
    };"""
MyExampleWidget(props={"message": "hi", "title": "Behaves like a tooltip"}, children=[' extra', ' children'])

## Adding state

Props cannot be changed by the component, they are considered pure input.

When you need the component to control state, you can add a trait ([see traitlets](https://traitlets.readthedocs.io/en/stable/using_traitlets.html)) to your widget class will (with `.tag(sync=True)` to make it sync to the frontend). For every trait added, you will receive a value and a setter to our props in the frontend.

For instance, if you add a trait called `message`, you will have a `message` and `setMessage` in your props.

In [ ]:
import ipyreact
from traitlets import Unicode

class MyExampleWidget(ipyreact.Widget):
    message = Unicode("Click me").tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({ message, setMessage }) {
        return <button onClick={() => setMessage('Clicked ⭐')}> {message} </button>;
    };"""
w = MyExampleWidget()
w

Every time the component calls `setMessage` the component will rerender itself with the new value for `message`, but will also synchronize the value to the Python side. 

If we change the value from the Python side, the value gets send to the frontend, and the component will render with the new `message` value.

In [ ]:
w.message = "Set from Python 🐍"

In [ ]:
# we can use this traitlets also as parameters
MyExampleWidget(message="Different initial value")

It's great that python will throw an error when the wrong type is given!

In [ ]:
# Note that traits can be type checked, this will result in an error because message is not a string (it is an int)
# w.message = 1

### ValueWidget

Since it is very common that a component controls a single value (e.g. any input component, such as text input, a slider etc) we made a special subclass of `ipyreact.Widget` called `ipyreact.ValueWidget` that already contains a `value` trait.

In many cases you do not even need to create a subclass, but can directly use the class to create an instance.

In [ ]:
import ipyreact
from traitlets import Unicode


# Although we can subclass, we don't need to in this case
# class MyExampleValueWidget(ipyreact.ValueWidget):
#     # we get a value trait for free
#     _esm = """
#     import * as React from "react";

#     export default function MyButton({ value, setValue }) {
#         return <button onClick={() => setValue('Clicked ⭐')}> {value} </button>;
#     };"""
# MyExampleValueWidget(value="Similar, but using the value/ValueWidget")


# We can simply create an instance of ValueWidget

ipyreact.ValueWidget(value="Similar, but using the value/ValueWidget",
    _esm="""
    import * as React from "react";

    export default function MyButton({ value, setValue }) {
        return <button onClick={() => setValue('Clicked ⭐')}> {value} </button>;
    };"""
)

The upside of using the `ValueWidget` is that it is a subclass of `ipywidgets.ValueWidget` and therefore can be used in [interact](https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html).

Having a standard name (`value`) for the trait can be useful, but might loss semantics in your specific case.

Note that you may be tempted to add in many traits, since it makes it easier to modify the state of the component. But be aware that for ever trait added, your props get both a `foo` and `setFoo`. Make sure you do not accidently pass the `setFoo` to your child elements, as they might not support it (e.g. button has no setFoo attribute).


## Adding events

Apart from the traits, the props trait, and the children, we also support events.

The events dict is a mapping from event name to an event handler. Event names can be native browser events, such as onClick on native elements (e.g. button), but they can also be custom events.

### Native events

Native browser events are always of the form `on<Name>`, for instance, the [click](https://developer.mozilla.org/en-US/docs/Web/API/Element/click_event) event will map to the `onClick` event name.

In [ ]:
def on_click(event_data):
    w.children = ["Clicked ⭐"]
    
w = ipyreact.Widget(_type="button",
                    children=["Click me"],
                    props={
                        "title": "Behaves like a tooltip",
                        "style": {"border": "5px solid orange"},
                        "class": "mybutton"
                    },
                    events={"onClick": on_click})
w

### Custom events

If you are creating your own component, you are free to name events anything you'd like. Note that event handlers can optionally take arguments.

In [ ]:
def on_my_click(new_label):
    w.children = [new_label]
    
    
w = ipyreact.Widget(children=["Click me"],
    events={"onMyClick": on_my_click},
    _esm="""
    import * as React from "react";

    export default function MyButton({ onMyClick, children }) {
        return <button onClick={() => onMyClick('Clicked ⭐')}> {...children} </button>;
    };
    """
)
w

It is also possible to add methods with a subclass, prefixed with `event_` that will automatically be available in the props as well.

In [ ]:
class MyButton(ipyreact.Widget):
    _esm = """
    import * as React from "react";

    export default function MyButton({ onMyClick, children }) {
        return <button onClick={() => onMyClick('Clicked ⭐')}> {...children} </button>;
    };
    """

    # the method name should match the name in the props
    def event_onMyClick(self, new_label):
        w.children = [new_label]

w = MyButton(children=["Click me"])
w

## Traitlet events / observe

Since traits can be [observed for changes](https://traitlets.readthedocs.io/en/stable/using_traitlets.html#observe) we can also add an event handler to state changes (instead of the event handler solution is the previous example).

Both solutions can be valid. Sometimes events go together with a state change, and observing a state change then makes sense. In cases where a pure event is emitted, that does not directly lead to a state change, this solution might not be the right one.

The example below does combine an event with a state change, and we therefore use the `@observe` decorator to handle further state changes.

In [ ]:
from traitlets import  Any, observe
from traitlets import Int, Any
import ipyreact

def is_prime_number(n):
    for i in range(2, n):
        if n % i == 0:
            return False
    return True


class PrimePythonWidget(ipyreact.Widget):
    message = Any("Click to test the next number").tag(sync=True)
    number = Int(0).tag(sync=True)

    @observe("number")
    def _observe_count(self, change):
        if is_prime_number(self.number):
            self.message = "Yes ✅ it is a prime number"
        else:
            self.message = "No ❌, not a primer number"
        # alternatively: 
        # self.props = {**self.props, message: ....}

    _esm = """
    import * as React from "react";

    // NOTE: we add setMessage, even though we do not use it, to avoid forwarding
    // it to button
    export default function({setNumber, number, message, setMessage, ...rest}) {
        return <div>
            <button onClick={() => setNumber(number + 1)} {...rest}>
                Testing {number} for prime
            </button>
            <br/>
            <span>{message}</span>
        </div>
    };"""


primepy = PrimePythonWidget(props={"class": "mybutton"})
primepy

Note that in this case, we have chosen to add `message` as a trait, instead of sending the `message` via the `props` trait. Since we also combine this with forwarding the rest of the props to the button, we *have* to take out the `setMessage` callback. If we do not, React will complain that the button element does not support the `setMessage` attribute.

## Components in files
Having the JavaScript components in python string variables is good for the beginning.  
That way the project is compact and there is no need of file switching.  

As examples are getting longer, the JavaScript components can be written in separate files.  
That way, you will also get JavaScript syntax hilighting.  

In [ ]:
import ipyreact
import pathlib

class WidgetFromFile(ipyreact.Widget):
    _esm = pathlib.Path("my_component.tsx").read_text()

WidgetFromFile()

If you don't want to re-run the python code after making changes to the file that contains the component, you can see changes happening immediately thanks to **hot-reloading**.
This requires `pip install watchdog`.  
Next, you replace the line  
`_esm = pathlib.Path("my_component.tsx").read_text()`  
with   
`_esm = pathlib.Path("my_component.tsx")`  

Now open `my_component.tsx`, change "Hello World" to "Hi there", and you will see that the changes are reflected immediately.

In [ ]:
import ipyreact
import pathlib

class WidgetFromFile(ipyreact.Widget):
    _esm = pathlib.Path("my_component.tsx") # <- this will not work in JupyterLite

WidgetFromFile()

### Importing external modules

Writing JSX code without having to compile/bundle is great, but so is using external libraries.

Ipyreact uses ES modules, which allows native importing of external libraries when written as an ES module.
In the example below, we use https://esm.sh/ which exposes many JS libraries as ES modules that
we can directly import.

In [ ]:
import ipyreact

ipyreact.ValueWidget(
    _esm="""
    import confetti from "https://esm.sh/canvas-confetti@1.6.0";
    import * as React from "react";

    export default function({value, setValue}) {
        return <button onClick={() => confetti() && setValue(value + 1)}>
            {value || 0} times confetti
        </button>
    };
    """
)

However, the above code now has a direct link to "https://esm.sh/canvas-confetti@1.6.0" which makes the code very specific to esm.sh.

To address this, we also support [import maps](https://developer.mozilla.org/en-US/docs/Web/HTML/Element/script/type/importmap) to 
write code more independant of where the modules come from.
You can provide an import map using `ipyreact.define_import_map`, which takes a dictionary of module names to urls or other modules. By default we support `react` and `react-dom` which is prebundled.

Apart from `react`, the default we provide is:

```python
define_import_map({
    "@mui/material": "https://esm.sh/@mui/material@5.11.10?external=react,react-dom",
    "@mui/material/": "https://esm.sh/@mui/material@5.11.10&external=react,react-dom/",
    "@mui/icons-material/": "https://esm.sh/@mui/icons-material/?external=react,react-dom",
    "canvas-confetti": "https://esm.sh/canvas-confetti@1.6.0?external=react,react-dom",
})

```

Which means we can now write our ConfettiButton as:


In [ ]:
import ipyreact

# note that this import_map is already part of the default
ipyreact.define_import_map({
    "canvas-confetti": "https://esm.sh/canvas-confetti@1.6.0",
})


ipyreact.ValueWidget(
    _esm="""
    import confetti from "canvas-confetti";
    import * as React from "react";

    export default function({value, setValue}) {
        return <button onClick={() => confetti() && setValue(value + 1)}>
            {value || 0} times confetti
        </button>
    };
    """
)

## Advanced

 * [Bundled ES modules (ant design example)](./antd/antd.ipynb)

## Optional
### Autocomplete

one more thing:  
Having **autocompletion in IDEs** is awesome!  
traitlets don't have that by default, but adding a `signature_has_traits` decorator will do the job!

In [ ]:
import ipyreact
from traitlets import Any, Unicode, Int, observe, signature_has_traits

@signature_has_traits
class MyExampleWidget(ipyreact.Widget):
    my_width = Int(23).tag(sync=True)
    _esm = """
    import * as React from "react";

    export default function MyButton({ my_width }) {
      return (
        <button
          style={{
            position: "relative",
            width: my_width,
            height: 30,
          }}
        >
          {" "}
          Width of {my_width} px{" "}
        </button>
      );
    }"""
MyExampleWidget(my_width=300)

And this screenshots shows that autocompletion works now:  
<img src="autocomplete_screenshot.png" width="400">

Now we want to **print a message at class initialization** that says "Button was initialized with width 300px!"  
That is possible with the following code pattern using calling the `super` method.

In [ ]:
import ipyreact
from traitlets import Int, signature_has_traits

# 🪄🪄🪄 this is an advanced example, feel free to skip 🪄🪄🪄

@signature_has_traits
class MyExampleWidget(ipyreact.Widget):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.print_welcome_message()

    def print_welcome_message(self):
        print(f"Button was initilized with width of {self.my_width}px ")

    my_width = Int(23).tag(sync=True)

    _esm = """
    import * as React from "react";

    export default function MyButton({ my_width }) {
    return (
        <button
        style={{
            position: "relative",
            width: my_width,
            height: 30,
        }}
        >
        {" "}
        Width of {my_width} px{" "}
        </button>
    );
    }"""

MyExampleWidget(my_width=200)